## **This code aims to clean all existing dfs**
- Appropiate date format (if needed)
- Creates Return column
- Interpolate NA
- Remove outliers
- Generate target column
- Create df's for each period

In [1]:
import pandas as pd
import os
import numpy as np
import sys
from datetime import timedelta

In [2]:
project_dir = "/home/jupyter-tfg2425paula/prediction_project_v3"
os.chdir(project_dir)

raw_data_dir = os.path.join(project_dir, "00_data/raw")
clean_data_dir = os.path.join(project_dir, "00_data/clean")

In [3]:
def appropiate_date_format(df, date_col_name, date_format="%d/%m/%y"):
    """ Converts date column to appropiate format
    """
    df[date_col_name] = pd.to_datetime(df[date_col_name], format=date_format)
    return df

def create_return_column(df, target_col_name):
    """ Generates Return column
    """
    df = df.copy()
    df[target_col_name] = pd.to_numeric(df[target_col_name], errors="coerce")
    df["Return"] = df[target_col_name].pct_change(fill_method="pad") * 100 
    df = df.drop(columns = target_col_name)
    
    return df

def remove_na(df):
    """
    Handles missing values in specified columns of a DataFrame using linear interpolation.
    Removes rows with missing values if they are at the beginning or end.
    """
    selected_cols = list(df.columns)
    na_method = "linear"
    df = df.iloc[1:].copy()
    df = df.iloc[:-1].copy()
    rows_with_na = df[df[selected_cols].isna().any(axis=1)]
    for col in selected_cols:
        df[col] = df[col].interpolate(na_method, limit_direction="both")

    return df

def replace_outliers_iqr(df, output_col):
    """
    Replaces outliers in a DataFrame using the IQR method with interpolated or mean/median values.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        selected_col (str): The column to process.
        threshold (float): The IQR threshold. Default is 1.5.
        method (str): The method to replace outliers ("interpolate", "mean", "median"). Default is "interpolate".
        
    Returns:
        pd.DataFrame: The DataFrame with outliers replaced.
    """
    method="linear"
    threshold = 2.5
    
    col = output_col
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
    
    extreme_outliers = df.loc[outliers, col]
    num_outliers = len(extreme_outliers)
    min_outlier = extreme_outliers.min() if not extreme_outliers.empty else None

    print(f"Number of outliers eliminated: {num_outliers}")
    print(f"Minimum extreme outlier value: {min_outlier}")
    
    df.loc[outliers, col] = np.nan
    df[col] = df[col].interpolate(method, limit_direction="both")

    return df

def generate_target_column(df, output_col):
    df['Target'] = (df[output_col].shift(-1) > 0).astype(float)
    
    return df
    
def filter_by_period(df, years):
    """
    Filters the dataframe rows based on the specified period.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing a 'Date' column.
    years (list): List of time periods as strings, e.g., ["15y", "10y", "5y", "2y"].
    
    Returns:
    dict: A dictionary of DataFrames filtered for each period.
    """
    # Ensure 'Date' column is in datetime format
    df["Date"] = pd.to_datetime(df["Date"])

    # Find the most recent date in the dataset
    max_date = df["Date"].max()

    # Dictionary to store filtered DataFrames
    filtered_dfs = {}

    # Iterate over the given periods
    for period in years:
        print(period)
        num_years = int(period[:-1])  # Extract numeric part
        cutoff_date = max_date - pd.DateOffset(years=num_years)  # Compute cutoff date

        # Filter DataFrame
        filtered_df = df[df["Date"] >= cutoff_date]

        filtered_df = filtered_df.reset_index(drop=True)
        
        # Store in dictionary
        filtered_dfs[period] = filtered_df

    return filtered_dfs

In [4]:
types_securities = ["technical"]
stocks = ['SPX']
years = ["2y", "5y", "10y", "15y"]


price_column = {"single_name": 0, 
                "options": 1,
                "technical": 0
}

file_names = {"single_name": "_Close.csv", 
                "options": "_options.csv",
                "technical": "_technical.csv"
}

seps = { "single_name": ";", 
                "options": ",",
                "technical": ","
}
decs = { "single_name": ",", 
                "options": ".",
                "technical": "."
}

for security_type in types_securities:
    for stock in stocks:
        output_folder = os.path.join(clean_data_dir, f"{security_type}/{stock}")
        os.makedirs(output_folder, exist_ok=True)
        intial_df = pd.read_csv(os.path.join(raw_data_dir, f"{security_type}/{stock}{file_names[security_type]}"), 
                                sep=seps[security_type], decimal=decs[security_type])
        print(f"{security_type}/{stock}{file_names[security_type]}")
        
        if security_type == "single_name":
            intial_df = appropiate_date_format(intial_df, "Date", date_format="%d/%m/%y")
        df = create_return_column(intial_df, f"{price_column[security_type]*str(stock+'_')}"+"Close")
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df = remove_na(df)        
        df = replace_outliers_iqr(df, "Return")
        df = generate_target_column(df, "Return")

        filtered_results = filter_by_period(df, years)
        
        for period, df_filtered in filtered_results.items():
            print(os.path.join(output_folder, f"{period}_data.csv"))
            df_filtered.to_csv(os.path.join(output_folder, f"{period}_data.csv"), index=False)

technical/SPX_technical.csv
Number of outliers eliminated: 94
Minimum extreme outlier value: -11.984055240393443
2y
5y
10y
15y
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/clean/technical/SPX/2y_data.csv
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/clean/technical/SPX/5y_data.csv


/tmp/ipykernel_53491/289240904.py:12: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in Series.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Return"] = df[target_col_name].pct_change(fill_method="pad") * 100
/tmp/ipykernel_53491/289240904.py:28: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(na_method, limit_direction="both")


/home/jupyter-tfg2425paula/prediction_project_v3/00_data/clean/technical/SPX/10y_data.csv
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/clean/technical/SPX/15y_data.csv
